## 1. Acquire data:

In [4]:
import kagglehub
path = kagglehub.dataset_download("mehmettahiraslan/customer-shopping-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.63M/1.63M [00:00<00:00, 6.38MB/s]

Extracting files...
Path to dataset files: C:\Users\hweinguyen\.cache\kagglehub\datasets\mehmettahiraslan\customer-shopping-dataset\versions\2


In [ ]:
# move data from path to the current path
!move "C:\Users\hweinguyen\.cache\kagglehub\datasets\mehmettahiraslan\customer-shopping-dataset\versions\2\customer_shopping_data.csv" "C:\Users\hweinguyen\Final Project\Programming"

        1 file(s) moved.


In [7]:
import pandas as pd

# read file
df = pd.read_csv('customer_shopping_data.csv')
df.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon


## 2. Check connection: 

In [2]:
import mysql.connector
from mysql.connector import Error

connection = None

try: 
  connection = mysql.connector.connect(host = 'customter-database.cxkmy8keu4t4.ca-central-1.rds.amazonaws.com',
                                       database = 'customer_database',
                                       user='admin',
                                       password='Quangminh0512',
                                       port='3306')
  if connection.is_connected():
    db_info = connection.get_server_info()
    print("MySQL Server info: ", db_info)

    cursor = connection.cursor()
    cursor.execute("select database();")
    record = cursor.fetchone()
    print("You're connected to database: ", record)
        
except Error as e:
  print("Error while connecting to MySQL", e)

finally:
  if connection and connection.is_connected():
    cursor.close()
    connection.close()
    print("MySQL connection is closed")


MySQL Server info:  8.0.39
You're connected to database:  ('customer_database',)
MySQL connection is closed


## 3. Clean data:

In [8]:
# check duplicate
df[df.duplicated()]

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall


In [9]:
# check null
df.isnull().any()

invoice_no        False
customer_id       False
gender            False
age               False
category          False
quantity          False
price             False
payment_method    False
invoice_date      False
shopping_mall     False
dtype: bool

In [10]:
# convert string of invoice_date to date type
df['invoice_date'] = pd.to_datetime(df['invoice_date'], format='%d/%m/%Y')

In [11]:
# sort the data from the oldest to newest
df.sort_values(by='invoice_date', ascending=True, inplace=True, ignore_index=True)
df.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I192911,C642829,Female,66,Food & Beverage,5,26.15,Credit Card,2021-01-01,Metrocity
1,I313757,C438201,Female,43,Clothing,4,1200.32,Credit Card,2021-01-01,Mall of Istanbul
2,I836951,C382440,Male,54,Clothing,3,900.24,Cash,2021-01-01,Metrocity
3,I159235,C312481,Male,58,Food & Beverage,4,20.92,Credit Card,2021-01-01,Mall of Istanbul
4,I115941,C105769,Male,34,Clothing,1,300.08,Cash,2021-01-01,Istinye Park


## 4. Create table and load data to cloud:

In [18]:
try:
    connection = mysql.connector.connect(
        host='customter-database.cxkmy8keu4t4.ca-central-1.rds.amazonaws.com',
        user='admin',
        password='Quangminh0512',
        database='customer_database',
        port='3306'
    )
    cursor = connection.cursor()

    # Create table Transactions:
    if connection.is_connected():
            create_table_query = """
            CREATE TABLE IF NOT EXISTS Transactions (
                TransactionID INT AUTO_INCREMENT PRIMARY KEY,
                invoice_no VARCHAR(255),
                customer_id VARCHAR(255),
                gender ENUM('Female', 'Male'),
                age INT,
                category VARCHAR(255),
                quantity INT,
                price DECIMAL(10, 2),
                payment_method ENUM('Cash', 'Debit Card', 'Credit Card'),
                invoice_date DATE,
                shopping_mall VARCHAR(255)
            );
            """
            cursor.execute(create_table_query)
            print("Table 'Transactions' created successfully.")

    # Insert data row by row into table Transactions:
    for _, row in df.iterrows():
        load_data_to_table = """
                        INSERT INTO Transactions 
                        (invoice_no, customer_id, gender, age, category, quantity, price, payment_method, invoice_date, shopping_mall)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s); 
                        """ 
        cursor.execute(load_data_to_table, 
                        (row['invoice_no'], row['customer_id'], row['gender'], row['age'], row['category'], row['quantity'], 
                         row['price'], row['payment_method'], row['invoice_date'], row['shopping_mall']))
    
    print("Data loaded successfully into table Transactions")

    connection.commit()
             
except Error as e:
    print("Error while loading data to cloud", e)
    
finally:
    if connection and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Table 'Transactions' created successfully.
Data loaded successfully into table Transactions
MySQL connection is closed
